In [1]:
import numpy as np

In [3]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import csv
import os.path
import time
import numpy as np

import tensorflow as tf

import gpr
import load_dataset
import nngp

tf.logging.set_verbosity(tf.logging.INFO)

hparams=''
experiment_dir='/tmp/nngp'
grid_path='./grid_data'
nonlinearity='relu'
depth=3
weight_var=1.79
bias_var=0.83

num_train=4000

num_eval=1000
seed=1234
save_kernel=False
dataset='mnist'
n_gauss=501
n_var=501
n_corr=500
max_var=100
max_gauss=10
use_fixed_point_norm=False
return_reults=False


def set_default_hparams(nl='relu', d=2, wv=1.3, bv=0.2):
  return tf.contrib.training.HParams(
      nonlinearity=nl, depth=d, weight_var=wv, bias_var=bv)


def do_eval(sess, model, x_data, y_data, save_pred=False):
  """Run evaluation."""

  gp_prediction, stability_eps = model.predict(x_data, sess)

  pred_1 = np.argmax(gp_prediction, axis=1)
  accuracy = np.sum(pred_1 == np.argmax(y_data, axis=1)) / float(len(y_data))
  mse = np.mean(np.mean((gp_prediction - y_data)**2, axis=1))
  pred_norm = np.mean(np.linalg.norm(gp_prediction, axis=1))
  tf.logging.info('Accuracy: %.4f'%accuracy)
  tf.logging.info('MSE: %.8f'%mse)

  if save_pred:
    with tf.gfile.Open(
        os.path.join(experiment_dir, 'gp_prediction_stats.npy'),
        'w') as f:
      np.save(f, gp_prediction)

  return accuracy, mse, pred_norm, stability_eps


def run_nngp_eval(hparams, run_dir):
  """Runs experiments."""

  tf.gfile.MakeDirs(run_dir)
  # Write hparams to experiment directory.
  with tf.gfile.GFile(run_dir + '/hparams', mode='w') as f:
    f.write(hparams.to_proto().SerializeToString())

  tf.logging.info('Starting job.')
  tf.logging.info('Hyperparameters')
  tf.logging.info('---------------------')
  tf.logging.info(hparams)
  tf.logging.info('---------------------')
  tf.logging.info('Loading data')

  # Get the sets of images and labels for training, validation, and
  # # test on dataset.
  if dataset == 'mnist':
    (train_image, train_label, valid_image, valid_label, test_image,
     test_label) = load_dataset.load_mnist(
         num_train=num_train,
         mean_subtraction=True,
         random_roated_labels=False)
  else:
    raise NotImplementedError

  tf.logging.info('Building Model')

  if hparams.nonlinearity == 'tanh':
    nonlin_fn = tf.tanh
  elif hparams.nonlinearity == 'relu':
    nonlin_fn = tf.nn.relu
  else:
    raise NotImplementedError

  with tf.Session() as sess:
    # Construct NNGP kernel
    nngp_kernel = nngp.NNGPKernel(
        depth=hparams.depth,
        weight_var=hparams.weight_var,
        bias_var=hparams.bias_var,
        nonlin_fn=nonlin_fn,
        grid_path=grid_path,
        n_gauss=n_gauss,
        n_var=n_var,
        n_corr=n_corr,
        max_gauss=max_gauss,
        max_var=max_var,
        use_fixed_point_norm=use_fixed_point_norm)

    # Construct Gaussian Process Regression model
    model = gpr.GaussianProcessRegression(
        train_image, train_label, kern=nngp_kernel)

    start_time = time.time()
    tf.logging.info('Training')

    # For large number of training points, we do not evaluate on full set to
    # save on training evaluation time.
    if num_train <= 5000:
      acc_train, mse_train, norm_train, final_eps = do_eval(
          sess, model, train_image[:num_eval],
          train_label[:num_eval])
      tf.logging.info('Evaluation of training set (%d examples) took '
                      '%.3f secs'%(
                          min(num_train, num_eval),
                          time.time() - start_time))
    else:
      acc_train, mse_train, norm_train, final_eps = do_eval(
          sess, model, train_image[:1000], train_label[:1000])
      tf.logging.info('Evaluation of training set (%d examples) took '
                      '%.3f secs'%(1000, time.time() - start_time))

    start_time = time.time()
    tf.logging.info('Validation')
    acc_valid, mse_valid, norm_valid, _ = do_eval(
        sess, model, valid_image[:num_eval],
        valid_label[:num_eval])
    tf.logging.info('Evaluation of valid set (%d examples) took %.3f secs'%(
        num_eval, time.time() - start_time))

    start_time = time.time()
    tf.logging.info('Test')
    acc_test, mse_test, norm_test, _ = do_eval(
        sess,
        model,
        test_image[:num_eval],
        test_label[:num_eval],
        save_pred=False)
    tf.logging.info('Evaluation of test set (%d examples) took %.3f secs'%(
        num_eval, time.time() - start_time))

  metrics = {
      'train_acc': float(acc_train),
      'train_mse': float(mse_train),
      'train_norm': float(norm_train),
      'valid_acc': float(acc_valid),
      'valid_mse': float(mse_valid),
      'valid_norm': float(norm_valid),
      'test_acc': float(acc_test),
      'test_mse': float(mse_test),
      'test_norm': float(norm_test),
      'stability_eps': float(final_eps),
  }

  record_results = [
      num_train, hparams.nonlinearity, hparams.weight_var,
      hparams.bias_var, hparams.depth, acc_train, acc_valid, acc_test,
      mse_train, mse_valid, mse_test, final_eps
  ]
  if nngp_kernel.use_fixed_point_norm:
    metrics['var_fixed_point'] = float(nngp_kernel.var_fixed_point_np[0])
    record_results.append(nngp_kernel.var_fixed_point_np[0])

  # Store data
  result_file = os.path.join(run_dir, 'results.csv')
  with tf.gfile.Open(result_file, 'a') as f:
    filewriter = csv.writer(f)
    filewriter.writerow(record_results)

  return metrics


def main(argv):
  del argv  # Unused
  hparams = set_default_hparams(nonlinearity,
                                        depth,
                                        weight_var,
                                        bias_var)
  tf.logging.info(hparams.nonlinearity)
  run_nngp_eval(hparams, experiment_dir)


if __name__ == '__main__':
  tf.app.run(main)

INFO:tensorflow:relu
INFO:tensorflow:Starting job.
INFO:tensorflow:Hyperparameters
INFO:tensorflow:---------------------
INFO:tensorflow:[('bias_var', 0.83), ('depth', 3), ('nonlinearity', 'relu'), ('weight_var', 1.79)]
INFO:tensorflow:---------------------
INFO:tensorflow:Loading data
Extracting /tmp/nngp/data/train-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/nngp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Building Model
INFO:tensorflow:Loaded interpolation grid from ./grid_data\grid_relu_ng501_ns501_nc500_mv100_mg10
Instructions for updating:
keep_dims is deprecated, use keepdims instead
INFO:tensorflow:Training
INFO:tensorflow:Computing Kernel
INFO:tensorflow:Computed K_DD in 22.270 secs
INFO:tensorflow:Computed L_DD in 5.264 secs
INFO:tensorflow:Using pre-computed Kernel
INFO:tensorflow:Did regression in 9.566 secs
INFO:tensorflow:Accuracy: 1.0000
INFO:tensorflow:MSE: 0.00000000
IN

SystemExit: 

C:\Users\delan\AppData\Local\Continuum\anaconda3\envs\dnngp\lib\site-packages\IPython\core\interactiveshell.py:3339: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [7]:
#size_train_iter = np.loadtxt('size_train_iter.txt')
size_train_iter = [10,20, 50, 100, 250, 500, 750, 1000, 1500, 2000]

In [8]:
from tqdm import tqdm

In [9]:
nngp_list = []

for ntrain in tqdm(size_train_iter):
    hparams=''
    experiment_dir='/tmp/nngp'
    grid_path='./grid_data'
    nonlinearity='relu'
    depth=3
    weight_var=1
    bias_var=0.2
    num_train=int(ntrain)
    num_eval=1000
    seed=1235
    save_kernel=False
    dataset='mnist'
    n_gauss=501
    n_var=501
    n_corr=500
    max_var=100
    max_gauss=10
    use_fixed_point_norm=False
    return_reults=False

    hparams = set_default_hparams(nonlinearity,
                                        depth,
                                        weight_var,
                                        bias_var)

    me = run_nngp_eval(hparams, experiment_dir)
    print(me)
    nngp_list += [me['test_acc']]
    np.savetxt('nngp_list3.txt', np.array(nngp_list))


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]

INFO:tensorflow:Starting job.
INFO:tensorflow:Hyperparameters
INFO:tensorflow:---------------------
INFO:tensorflow:[('bias_var', 0.2), ('depth', 3), ('nonlinearity', 'relu'), ('weight_var', 1)]
INFO:tensorflow:---------------------
INFO:tensorflow:Loading data
Extracting /tmp/nngp/data/train-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/nngp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Building Model
INFO:tensorflow:Loaded interpolation grid from ./grid_data\grid_relu_ng501_ns501_nc500_mv100_mg10
INFO:tensorflow:Training
INFO:tensorflow:Computing Kernel
INFO:tensorflow:Computed K_DD in 21.485 secs
INFO:tensorflow:Computed L_DD in 0.340 secs
INFO:tensorflow:Using pre-computed Kernel
INFO:tensorflow:Did regression in 2.099 secs
INFO:tensorflow:Accuracy: 1.0000
INFO:tensorflow:MSE: 0.00000000
INFO:tensorflow:Evaluation of training set (10 examples) took 25.538 secs
INFO:tensorflow:Validation

 10%|████████▎                                                                          | 1/10 [00:35<05:18, 35.42s/it]

{'train_acc': 1.0, 'train_mse': 5.224652530008587e-19, 'train_norm': 0.9486832746534055, 'valid_acc': 0.58, 'valid_mse': 0.06469660793470669, 'valid_norm': 0.36759831979566815, 'test_acc': 0.56, 'test_mse': 0.06649879415662864, 'test_norm': 0.36817324517112043, 'stability_eps': 1e-10}
INFO:tensorflow:Starting job.
INFO:tensorflow:Hyperparameters
INFO:tensorflow:---------------------
INFO:tensorflow:[('bias_var', 0.2), ('depth', 3), ('nonlinearity', 'relu'), ('weight_var', 1)]
INFO:tensorflow:---------------------
INFO:tensorflow:Loading data
Extracting /tmp/nngp/data/train-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/nngp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Building Model
INFO:tensorflow:Loaded interpolation grid from ./grid_data\grid_relu_ng501_ns501_nc500_mv100_mg10
INFO:tensorflow:Training
INFO:tensorflow:Computing Kernel
INFO:tensorflow:Computed K_DD in 24.052 secs
INFO:tenso

 20%|████████████████▌                                                                  | 2/10 [01:14<04:52, 36.54s/it]

...

{'train_acc': 1.0, 'train_mse': 1.477449269531301e-18, 'train_norm': 0.9486832751852522, 'valid_acc': 0.925, 'valid_mse': 0.021502433981015267, 'valid_norm': 0.785728066837612, 'test_acc': 0.911, 'test_mse': 0.0238795439717183, 'test_norm': 0.7539945768227921, 'stability_eps': 1e-10}
INFO:tensorflow:Starting job.
INFO:tensorflow:Hyperparameters
INFO:tensorflow:---------------------
INFO:tensorflow:[('bias_var', 0.2), ('depth', 3), ('nonlinearity', 'relu'), ('weight_var', 1)]
INFO:tensorflow:---------------------
INFO:tensorflow:Loading data
Extracting /tmp/nngp/data/train-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/nngp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Building Model
INFO:tensorflow:Loaded interpolation grid from ./grid_data\grid_relu_ng501_ns501_nc500_mv100_mg10
INFO:tensorflow:Training
INFO:tensorflow:Computing Kernel
INFO:tensorflow:Computed K_DD in 39.143 secs
INFO:tensor

 90%|██████████████████████████████████████████████████████████████████████████▋        | 9/10 [07:13<00:55, 55.03s/it]

{'train_acc': 1.0, 'train_mse': 1.6326353791895778e-18, 'train_norm': 0.9486832752278775, 'valid_acc': 0.933, 'valid_mse': 0.018664141176674042, 'valid_norm': 0.8019808173892133, 'test_acc': 0.922, 'test_mse': 0.020778420155534234, 'test_norm': 0.7664372348602985, 'stability_eps': 1e-10}
INFO:tensorflow:Starting job.
INFO:tensorflow:Hyperparameters
INFO:tensorflow:---------------------
INFO:tensorflow:[('bias_var', 0.2), ('depth', 3), ('nonlinearity', 'relu'), ('weight_var', 1)]
INFO:tensorflow:---------------------
INFO:tensorflow:Loading data
Extracting /tmp/nngp/data/train-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/nngp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/nngp/data/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Building Model
INFO:tensorflow:Loaded interpolation grid from ./grid_data\grid_relu_ng501_ns501_nc500_mv100_mg10
INFO:tensorflow:Training
INFO:tensorflow:Computing Kernel
INFO:tensorflow:Computed K_DD in 41.961 secs
INFO:te

100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [08:25<00:00, 50.60s/it]

{'train_acc': 1.0, 'train_mse': 1.9192044076973876e-18, 'train_norm': 0.9486832750762437, 'valid_acc': 0.94, 'valid_mse': 0.017090590461514288, 'valid_norm': 0.8111912496108032, 'test_acc': 0.928, 'test_mse': 0.019314253402563205, 'test_norm': 0.7814530276486508, 'stability_eps': 1e-10}
